In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import datetime as dt
import plotly.express as px

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)

import scripts.functions as fn
from scripts.indicators import supertrend
#from scripts.indicators import resample
from binance.client import Client

c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [2]:
#Funcion para descarga de velas
client = Client()
def get_klines(symbol,interval,start,end):
    print('Descargando velas',symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

Obtener el top30 de Criptos, excluyendo BTC, ETH, BNB y StableCoins 

In [3]:
top30 = [
        'XRPUSDT',
        'SOLUSDT',
        'TRXUSDT',
        'DOGEUSDT',
        'ADAUSDT',
        'WBTCUSDT',
        'BCHUSDT',
        'SUIUSDT',
        'LINKUSDT',
        'XLMUSDT',
        'AVAXUSDT',
        'SHIBUSDT',
        'LTCUSDT',
        'HBARUSDT',
        'DOTUSDT',
        'UNIUSDT',
        'PEPEUSDT',
        'AAVEUSDT',
        'APTUSDT',
        'NEARUSDT',
        'ICPUSDT',
        'ETCUSDT',
        'VETUSDT',
        'ATOMUSDT',
        'FETUSDT',
        'FILUSDT',
        'WLDUSDT',
        'ALGOUSDT',
        'NEXOUSDT',
        'OPUSDT',
        ]
top30_file = './backtest/klines/1h01/top30_TOP30_1h01_2024-05-01_2025-05-31.DataFrame'
if not os.path.exists(top30_file):
    """
    import requests
    top30 = []
    # 1. Obtener monedas desde CoinGecko con market cap
    coingecko_url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 250,
        "page": 1
    }
    cg_data = requests.get(coingecko_url, params=params).json()
    # 2. Obtener lista de pares de Binance
    binance_info = requests.get("https://api.binance.com/api/v3/exchangeInfo").json()
    symbols = [s["symbol"] for s in binance_info["symbols"] if s["quoteAsset"] == "USDT" and s["status"] == "TRADING"]
    # 3. Cruzar datos: CoinGecko vs Pares Binance USDT
    usdt_marketcap_ranking = []
    for coin in cg_data:
        
        symbol_binance = coin['symbol'].upper() + "USDT"
        if symbol_binance in symbols:
            usdt_marketcap_ranking.append({
                "cg_id": coin['id'],
                "name": coin["name"],
                "symbol": symbol_binance,
                "market_cap": coin["market_cap"],
                "current_price": coin["current_price"]
            })

    # 4. Ordenar por market cap
    usdt_marketcap_ranking = sorted(usdt_marketcap_ranking, key=lambda x: x["market_cap"], reverse=True)

    # 5. Mostrar top 10
    for i, coin in enumerate(usdt_marketcap_ranking[:55], 1):
        top30.append(coin['symbol'])

    """

    

    #len(top30)

Preparando la SMA del Top30

In [4]:

if not os.path.exists(top30_file):
    SMA_PERIODS = 60
    def calculate_4h_sma(df):
        df_4h_points = df[df['datetime'].dt.hour % 4 == 0].copy()

    data = pd.DataFrame(columns=['datetime'])
    symbols_len = len(top30)
    symbols_rec = 0
    for symbol in top30:
        symbols_rec += 1
        klines_file  = f'./backtest/klines/1h01/top30_{symbol}_1h01_2024-05-01_2025-05-31.DataFrame'
        with open(klines_file, 'rb') as file:
            df = pickle.load(file)
            print(f'Procesando {symbol}    {symbols_rec}/{symbols_len}       ',end='\r')

            if len(data)==0:
                data['datetime'] = df['datetime']
                
            df.set_index('datetime',inplace=True)
            # Paso 1: Obtener los precios de cierre de las velas de 4H ya completadas
            # Usamos resample('4H').last() para obtener el último precio de cierre de cada bloque de 4 horas.
            close_4h = df['close'].resample('4H').last()

            # Paso 2: Definir la función que hará el cálculo para cada vela de 1H
            def calculate_intra_bar_sma(row, series_4h_closes, period):
                """
                Calcula la SMA de 4H para una vela de 1H específica.
                
                Args:
                    row (pd.Series): La fila del dataframe de 1H. El índice debe ser el timestamp.
                    series_4h_closes (pd.Series): La serie con los cierres de las velas de 4H.
                    period (int): El período de la SMA (en este caso, 60).
                """
                current_time = row.name
                current_close = row['close']
                
                # Identificar el inicio del bloque de 4H al que pertenece la vela actual
                current_4h_block_start = current_time.floor('4H')
                
                # Obtener los cierres de las velas de 4H que se completaron ANTES del bloque actual
                previous_4h_closes = series_4h_closes[series_4h_closes.index < current_4h_block_start]
                
                # Necesitamos los últimos (periodo - 1) cierres, es decir, 59
                if len(previous_4h_closes) < period - 1:
                    return np.nan # No hay suficientes datos históricos
                    
                last_59_closes = previous_4h_closes.tail(period - 1)
                
                # Creamos una serie con el cierre actual para concatenarlo
                current_value_series = pd.Series([current_close])
                
                # Combinamos los 59 cierres históricos con el cierre de la vela de 1H actual
                values_for_sma = pd.concat([last_59_closes, current_value_series])
                
                # Calculamos y devolvemos la media
                return values_for_sma.mean()

            # Paso 3: Aplicar la función a cada fila del DataFrame de 1H
            # Esto puede tardar un poco en dataframes muy grandes, pero es la forma más precisa de simularlo.

            df['sma_4h'] = df.apply(
                calculate_intra_bar_sma, 
                axis=1, # Aplicar por filas
                series_4h_closes=close_4h, 
                period=60
            )

            df.reset_index(inplace=True)
            data[f'{symbol}_c'] = df['close']
            data[f'{symbol}_m'] = df['sma_4h']
            data[f'{symbol}_ta'] = np.where(df['close']>df['sma_4h'],1,0)



Calculando el sentimiento de mercado con el top30 sobre la sma60 de cada symbol

In [5]:
if not os.path.exists(top30_file):
    ta_columns = []
    for symbol in top30:
        ta_columns.append(f'{symbol}_ta')

    data['ta_sum'] = data[ta_columns].sum(axis=1)
    data['breadth'] = (data['ta_sum']/len(ta_columns))*100

    with open(top30_file, 'wb') as f:
        pickle.dump(data, f)
    print('Grabando archivo')
else:
    print('Leyendo archivo')
    with open(top30_file, 'rb') as file:
        data = pickle.load(file)

Leyendo archivo


Calculando Buy & Sell de acuerdo al breadth

In [6]:
symbol = 'BTCUSDT'
klines_file  = f'./backtest/klines/1h01/top30_{symbol}_1h01_2024-05-01_2025-05-31.DataFrame'
with open(klines_file, 'rb') as file:
    df = pickle.load(file)

df['breadth'] = data['breadth']

df['buy'] = np.where((df['breadth']!=0) & (df['breadth'].shift()==0),df['close'],None)
df['sell'] = np.where((df['breadth']!=100)& (df['breadth'].shift()==100),df['close'],None)

df = supertrend(df)

In [7]:
df_to_show = df #[0:20]
df_to_show['price'] = df_to_show['close']
indicators = [
     {'col': 'st_high','color': 'red','row': 1, 'mode':'markers',},
     {'col': 'st_low','color': 'green','row': 1, 'mode':'markers',},
        ]
   
indicators_out = [
     {'col': 'breadth','color': 'white','row': 1, 'mode':'lines',},
        ]

events = [
       {'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle' }, 
       {'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle' }, 
     ]


fig = fn.ohlc_chart(df_to_show,show_volume=False,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()


Creando indicador de top30 en el momento actual

In [8]:

totalAbove = 0
symbols_len = len(top30)
symbols_rec = 0
for symbol in top30:
    symbols_rec += 1
    klines = client.get_historical_klines(symbol=symbol, interval='4h', limit=60)
    print(f'Procesando {symbol}    {symbols_rec}/{symbols_len}       ',end='\r')
    df = pd.DataFrame(klines)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['close'] = df['close'].astype('float')
    mean = df['close'].mean()
    close = df.iloc[-1]['close']
    if close>mean:
        totalAbove += 1

print('\nResultado')
print(totalAbove)

Procesando OPUSDT    30/30         
Resultado
2
